In [200]:
import os

In [201]:
#now we have to change the directory to the root folder
%pwd
#so nnow we are in the research folder, but everything we have to perform in the root folder

'c:\\Users\\Nehal\\Desktop\\Text-Summarizer-final'

In [202]:
#to go back one folder
os.chdir("c://Users/Nehal/Desktop/Text-Summarizer-final")
%pwd
#now we are in the root folder

'c:\\Users\\Nehal\\Desktop\\Text-Summarizer-final'

In [203]:
from dataclasses import dataclass
from pathlib import Path

#now define a decorator
#this is not a python actual class, it is a data class, you can mention your variables here
#this eentire thing is an entity, it is the return type of a function
#whenever we create data ingestion configuration, at the time, dataingestionclass will return these variables
#we can take these variables and use it, when we call root_dir, it will automatically return that path mentioned in yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path

In [204]:
#import all the constants
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [205]:
from pathlib import Path
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Path("config/config.yaml"),
        params_filepath=PARAMS_FILE_PATH
    ):
        #it will read this yaml file and we can access all the variables
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
            
        )
        return data_ingestion_config
    #hence we have created our custom datatype of any function using dataclass we call it entity
    #now we can easily call the dataIngestion configuration and access everything
    

In [206]:
#now creating components
import os 
import urllib.request as request # to download data from url
import zipfile #to unzip the data
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [207]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
                
            )
            logger.info(f"{filename}downloaded with the following info : \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
            
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [208]:
#Now component creation is done, now we have to create a pipeline
#first we will handle the try except block, we will initialize configuration manager,
#we will take the data ingestion configuration and store it 
#then we will call the data ingestion components and it will take those data ingestion configuration,
# then we will execute those two methods, download the data and extract the zip file
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

{2024-09-13 20:45:33,306: INFO: common: yaml file: config\config.yaml loaded successfully}
{2024-09-13 20:45:33,308: INFO: common: yaml file: params.yaml loaded successfully}
{2024-09-13 20:45:33,309: INFO: common: Directory created at artifacts}
{2024-09-13 20:45:33,310: INFO: common: Directory created at artifacts/data_ingestion}
{2024-09-13 20:45:48,258: INFO: 2981561069: artifacts/data_ingestion/data.zipdownloaded with the following info : 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BF0C:35E03B:A9691:C61D9:66E45715
Accept-Ranges: bytes
Date: Fri, 13 Sep 2024 15:15:36 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10222-MAA
X-Cache: 

In [209]:
from pathlib import Path

# Convert the string into a Path object
config_path = Path("config/config.yaml")

# Now you can use it as a Path object
print(config_path)  # Output: config/config.yaml
print(config_path.exists())  # Check if the file exists


config\config.yaml
True
